In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re
url_pattern = r'https?://\S+|www\.\S+'

In [12]:
BASE_URL = 'https://cyberpolice.gov.ir'
CATEGORY = f'{BASE_URL}/media_knowledge/c?cat=43'
CATEGORY = 'https://cyberpolice.gov.ir/media_knowledge/c?q=views/ajax&title=&cat=All&created=&created_1=&page=0'

In [3]:
def create_driver(headless = True):
    options = Options()
    options.headless = headless
    options.add_argument('ignore-certificate-errors')
    options.add_argument("--start-maximized")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    return webdriver.Chrome("/usr/bin/chromedriver", options=options)

In [4]:
def get_post_content(href):
    global driver
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[1]) 
    driver.get(href)
    data = driver.execute_script("""
    return document.querySelector(".row.bs-1col-stacked.node").innerText;""")
    driver.close()
    return data

In [5]:
def get_posts(url,contents=[],i=1):
    global driver
    print('page_'+str(i))
    base_window = driver.current_window_handle

    driver.get(url)
    posts = driver.execute_script("""
    return Array.from(document.querySelectorAll(".view-content h2>a")).map(
            item=>item.getAttribute('href')
        ) 
            """)
    next_page = driver.execute_script("""return document.querySelector('a[rel="next"]')""")

    posts = list(map(lambda post:f'{BASE_URL}/{post}',posts))
    for post in posts:
        _ = get_post_content(post)
        DT.loc[len(DT.index)] = _
        contents.append(_)
        driver.switch_to.window(base_window)
      
    if next_page:
        i+=1
        url = next_page.get_attribute("href")
        get_posts(url,contents,i)    
    return contents

In [ ]:
DT = pd.DataFrame(columns=['body'])
driver = create_driver(False)

driver.get(CATEGORY)

# base_window = driver.current_window_handle

# driver.switch_to.window(base_window)
get_posts(CATEGORY,[])  
DT    
driver.close()

/tmp/ipykernel_11122/4278637175.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = headless
/tmp/ipykernel_11122/4278637175.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome("/usr/bin/chromedriver", options=options)


page_1
page_2
page_3
page_4
page_5


In [12]:
# DT.to_csv('./contents/cyper_02.csv')

# Insert Posts

In [6]:
BASE_URL = 'http://saha.moi.ir/'


USERNAME = '1270609726'
PASSWORD = '093678900450'

USERNAME = '1741995108'
PASSWORD = '4NpzuFqdEyCDv6T'

In [7]:
def insert_post(title,body):
    
   
    # select post tab
    driver.find_element(By.CSS_SELECTOR, """
    #js_main_feed_holder > div.activity_feed_form_share > ul > li:nth-child(4) > a
    """).click()
    time.sleep(2)
   
    # select label
    driver.execute_script("""
    document.querySelector("#feed_new_add_labels_box").innerHTML = `<input type="text" id="feed_labels_inline_search" readonly="" autocomplete="off" placeholder="*حوزه فعالیت مرتبط با پست..." spmhbindable="">
    <ul id="feed_labels_inline_search_result">

    </ul>
    <div id="feed_new_add_labels_selected"><div id="new_feed_labels_selected_item_113" class="new_feed_label_item_glob_cls"><label>شناخت و آسیب شناسی در حوزه پدافند غیر عامل</label><div onclick="removeFeedSelectedLabel(113)">x</div></div></div>
    <div id="feed_new_add_labels_selected_val"><input type="hidden" id="feed_labels_selected_hide_item_113" name="feedLabels[]" value="113"></div>`""")
    
    # select keywords
    driver.execute_script("""
    document.querySelector("#feed_new_add_keywords_box").innerHTML = `<input type="text" id="feed_keywords_inline_search" autocomplete="off" placeholder="انتخاب کلمه کلیدی..." spmhbindable=""><button onclick="$.ajaxCall('feed.addFeedKeyWord', ('kw='+$('#feed_keywords_inline_search').val())+prepareKeywordParams()); return false;">+ افزودن</button>
    <ul id="feed_keywords_inline_search_result"></ul>
    <div id="feed_new_add_keywords_selected"><div class="new_feed_keyword_item_glob_cls" id="new_feed_keywords_selected_item_2060"><div>پدافند سایبری</div><div onclick="removeFeedSelectedKeyWord(2060)">x</div></div><div class="new_feed_keyword_item_glob_cls" id="new_feed_keywords_selected_item_2059"><div>امنیت سایبری</div><div onclick="removeFeedSelectedKeyWord(2059)">x</div></div><div class="new_feed_keyword_item_glob_cls" id="new_feed_keywords_selected_item_912"><div>جنگ سایبری</div><div onclick="removeFeedSelectedKeyWord(912)">x</div></div></div>
    <div id="feed_new_add_keywords_selected_val"><input type="hidden" name="feedKeywords[]" value="2060" id="feed_keywords_selected_hide_item_2060"><input type="hidden" name="feedKeywords[]" value="2059" id="feed_keywords_selected_hide_item_2059"><input type="hidden" name="feedKeywords[]" value="912" id="feed_keywords_selected_hide_item_912"></div>`""")
    
    
    driver.execute_script("""
        titleINPUT = document.querySelector('input[name="val[blog_title]"]')
        titleINPUT.parentElement.click()
        titleINPUT.value = arguments[0]
        titleINPUT.parentElement.click()
        
        textareaINPUT = document.querySelector('textarea[name="val[status_info]"]')
        textareaINPUT.parentElement.click()
        textareaINPUT.value = arguments[1]
        textareaINPUT.click()
    
        
        
    """,title,body)
#     document.querySelector('#activity_feed_submit').click()
    driver.find_element(By.CSS_SELECTOR, """
        input[name="val[blog_title]"]
    """).send_keys(' ')
    driver.find_element(By.CSS_SELECTOR, """
    .activity_feed_form_button_position_button
    """).click()
    time.sleep(10)
#     driver.execute_script("document.querySelector('#activity_feed_submit').click()")
#     driver.execute_script("document.querySelector('#activity_feed_submit').click()")
#     driver.execute_script(f"""document.querySelector('#activity_feed_textarea_status_info').value = ``""")
#     driver.find_element(By.CSS_SELECTOR, "#activity_feed_textarea_status_info").send_keys(body)

    
#     driver.execute_script("document.querySelector('#activity_feed_submit').click()")
#     time.sleep(5)
    driver.get(BASE_URL)

In [8]:
DT = pd.read_csv('./contents/cyper_02.csv')
DT['title'] = DT['body'].str.split('\n').str[0]

In [9]:
len(DT)

123

In [11]:
driver = create_driver(False)
driver.maximize_window()

driver.get(BASE_URL)
driver.find_element(By.ID, "login").send_keys(USERNAME)
driver.find_element(By.ID, "password").send_keys(PASSWORD)
driver.find_element(By.ID, "loginbtn").click()

for index in range(100,115):
    post = DT.iloc[index]
    
    title_01 = re.sub(url_pattern, '', f"{post['title']} - بخش اول") 
    title_02 = re.sub(url_pattern, '', f"{post['title']} - بخش دوم") 
    
    post_01 = '\n'.join(post['body'].split('\n')[:len(post['body'].split('\n'))//2])
    post_02 = '\n'.join(post['body'].split('\n')[len(post['body'].split('\n'))//2:])
    
    insert_post(title_01, post_01)
    insert_post(title_02, post_02)
    
    print('post '+str(index)+' inserted ...')

driver.close()

/tmp/ipykernel_11122/4278637175.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = headless
/tmp/ipykernel_11122/4278637175.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome("/usr/bin/chromedriver", options=options)


post 100 inserted ...
post 101 inserted ...
post 102 inserted ...
post 103 inserted ...
post 104 inserted ...
post 105 inserted ...
post 106 inserted ...
post 107 inserted ...
post 108 inserted ...
post 109 inserted ...
post 110 inserted ...
post 111 inserted ...
post 112 inserted ...
post 113 inserted ...
post 114 inserted ...
